In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [84]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, Dense, Flatten, Conv1D, concatenate, Activation, LSTM, Dropout, Reshape, Lambda
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers, Model, Sequential
from keras.models import load_model
from sklearn import metrics
import numpy as np
from conllu import parse, parse_tree
from pathlib import Path
import os
from collections import OrderedDict
import bz2
import json
import dill

In [4]:
%aimport parsing
from parsing import Parser, Parse

%aimport helpers
from helpers import read_embeddings, ROOT, clean_deprel

In [5]:
vec_filename = "ubercorpus.lowercased.tokenized.300d.bz2"

In [6]:
data_dir = Path.home() / "repos/UD_Ukrainian-IU"

with list(data_dir.glob("*train*"))[0].open() as f:
    data = f.read()
trees = parse(data)

with list(data_dir.glob("*test*"))[0].open() as f:
    test_data = f.read()
test_trees = parse(test_data)

In [7]:
def build_vocabulary(trees, form="form"):
    word_index = {}
    pos_index = {}
    dep_index = {}
    for tree in trees:
        for word in tree:
            deprel = word["deprel"]
            word_id = len(word_index)+1
            pos_id = len(pos_index)+1
            dep_id = len(dep_index)+1
            word_t = word[form].lower()
            word_pos = word["upostag"]
            word_index[word_t] = word_index.get(word_t, word_id)
            pos_index[word_pos] = pos_index.get(word_pos, pos_id)
            dep_index[deprel] = dep_index.get(deprel, dep_id)

    word_index[ROOT["form"]] = len(word_index)+1
    pos_index[ROOT["upostag"]] = len(pos_index)+1
    return word_index, pos_index, dep_index

In [8]:
def get_data(trees, parser):
    o_labels = []
    o_features = []
    for tree in trees:
        labels, features, _ = parser.parse(tree)
        o_labels.extend(labels)
        o_features.extend(features)
    return o_labels, o_features

In [9]:
word_index, pos_index, dep_index = build_vocabulary(trees+test_trees)

In [10]:
word_2_vec, ndim, _ = read_embeddings(filename=vec_filename, word_index=word_index)

In [11]:
DEFAULT_VEC = np.zeros(ndim, np.float32)

In [12]:
embedding_matrix = np.zeros((len(word_index)+1, ndim))
for word, i in word_index.items():
    embedding_matrix[i] = word_2_vec.get(word, DEFAULT_VEC)

In [13]:
 def get_stack_context(depth, stack, data):
        if depth >= 3:
            return data[stack[-1]["id"]], data[stack[-2]["id"]], data[stack[-3]["id"]]
        elif depth >= 2:
            return data[stack[-1]["id"]], data[stack[-2]["id"]], 0
        elif depth == 1:
            return data[stack[-1]["id"]], 0, 0
        else:
            return 0, 0, 0

In [14]:
def get_buffer_context(k, buffer, data):
        if k >= 3:
            return data[buffer[0]["id"]], data[buffer[1]["id"]], data[buffer[2]["id"]]
        elif k >= 2:
            return data[buffer[0]["id"]], data[buffer[1]["id"]], 0
        elif k == 1:
            return data[buffer[0]["id"]], 0, 0
        else:
            return 0, 0, 0

In [15]:
def get_parse_context(word, deps, data, left=True):
    if not word or word == -1:
        return 0, (0, 0), (0, 0)
    deps = deps[word["id"]]
    num = len(deps)
    if not num:
        return num, (0, 0), (0, 0)
    elif num==1:
        return num, (data[deps[0][0]], deps[0][1]), (0, 0)
    else:
        temp = sorted(deps, key=lambda x: x[0], reverse=left)
        return num, (data[deps[0][0]], deps[0][1]), (data[deps[1][0]], deps[1][1])

In [16]:
def feature_builder(stack, queue, tree, parse=None, word_index=word_index, 
                    pos_index=pos_index, dep_index=dep_index, form="form",
                    log=0):
    words = []
    tags = []
    deps = []
    depth = len(stack)
    q_len = len(queue)
    if ROOT not in tree:
        tree = [ROOT, *tree]
    
    s0, s1, s2 = get_stack_context(depth, stack, tree)
    q0, q1, q2 = get_buffer_context(q_len, queue, tree)
    
    # Left two child of the top stack
    Ns0l, s0l1, s0l2 = get_parse_context(s0, parse.lefts, tree, left=True)  
    # Right two child of the top stack
    Ns0r, s0r1, s0r2 = get_parse_context(s0, parse.rights, tree, left=False)
    # Left two child of the second element on stack
    Ns1l, s1l1, s1l2 = get_parse_context(s1, parse.lefts, tree, left=True)
    # Left two child of the second element on stack
    Ns1r, s1r1, s1r2 = get_parse_context(s1, parse.rights, tree, left=False)
    
    _, s0l1l1, _ = get_parse_context(s0l1[0], parse.lefts, tree, left=True)
    _, s0r1r1, _ = get_parse_context(s0r1[0], parse.rights, tree, left=False)
    _, s1l1l1, _ = get_parse_context(s1l1[0], parse.lefts, tree, left=True)
    _, s1r1r1, _ = get_parse_context(s1r1[0], parse.rights, tree, left=False)
    
    if log:
        if s0:
            print(s0["form"], "id =", s0["id"], parse.lefts[s0["id"]],  parse.rights[s0["id"]])
            print()
            print("Top stack (rights): ", s0r1, s0r2)
            print()
            print("Right for right: ", s0r1r1)
            print()
            print("Top stack (lefts): ", s0l1, s0l2)
            print()
            print("Left for left: ", s0l1l1)
            print()
            print()
        print("="*20)
        if s1:
            print(s1["form"], "id =", s1["id"], parse.lefts[s1["id"]],  parse.rights[s1["id"]])
            print()
            print("Second stack (rights): ", s1r1, s1r2)
            print()
            print("Right for right: ", s1r1r1)
            print()
            print("Second stack (lefts): ", s1l1, s1l2)
            print()
            print("Left for left: ", s1l1l1)
            print()
            print()
        print("="*20)
    deps = [dep_index.get(s0l1[-1], 0), dep_index.get(s0l2[-1], 0),
            dep_index.get(s0r1[-1], 0), dep_index.get(s0r2[-1], 0),
            dep_index.get(s1l1[-1], 0), dep_index.get(s1l2[-1], 0),
            dep_index.get(s1r1[-1], 0), dep_index.get(s1r2[-1], 0),
            dep_index.get(s0l1l1[-1], 0), dep_index.get(s0r1r1[-1], 0),
            dep_index.get(s1l1l1[-1], 0), dep_index.get(s1r1r1[-1], 0)]
    
    for x in [s0, s1, s2, q0, q1, q2, 
              s0l1[0], s0l2[0], s0r1[0], s0r2[0], 
              s1l1[0], s1l2[0], s1r1[0], s1r2[0],
              s0l1l1[0], s0r1r1[0], s1l1l1[0], s1r1r1[0]]:
        if x:
            word = x[form].lower() if x["id"] else "ROOT"
            word_idx = word_index.get(word)
            pos_idx = pos_index.get(x["upostag"])
            words.append(word_idx)
            tags.append(pos_idx)
        else:
            words.append(x)
            tags.append(x)
    
    dist = s0["id"] - q0["id"] if q0 and s0 else 0
    nums = [dist, Ns0l, Ns0r, Ns1r, Ns1l]
    features = [*words, *tags, *deps, *nums]
    return features, len(words), len(tags), len(deps), len(nums)

In [85]:
N = 5

In [18]:
def chunks(l, n=N):
    for i in range(0, len(l)-n+1):
        yield l[i:i + n]

In [59]:
def get_data(trees, parser, feature_extractor):
    o_labels = []
    o_features = []
    k = 0
    for tree in trees:
        labels, features, _, n_w, n_t, n_d, n_num = parser.parse(tree, feature_extractor=feature_extractor)
        feature_chunks = list(chunks(features))
        label_chunks = list(chunks(labels))
        o_labels.extend(label_chunks)
        o_features.extend(feature_chunks)
        if tree in test_trees:
           k += len(feature_chunks)
    return o_labels, o_features, n_w, n_t, n_d, n_num, k

In [60]:
parser = Parser()

In [61]:
labels, features, n_w, n_t, n_d, n_num, k = get_data(trees+test_trees, parser, feature_builder)
n_unq = len(np.unique([el for x in labels for el in x]))
label_index = parser.label_index.copy()
#X = np.asarray(features)
X = pad_sequences(sequences=features, maxlen=N, padding="post")
#y = to_categorical(labels)
y = to_categorical(pad_sequences(labels, padding="post", value=n_unq))

In [63]:
n_train = X.shape[0]-k#
#n_train = 154709

In [64]:
X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

In [67]:
word_embedding_layer = Embedding(len(word_index)+1,
                            ndim,
                            weights=[embedding_matrix],
                            input_shape=(N,n_w),
                            trainable=0,
                            #mask_zero=True
                            #input_length=n_w
                           )

In [68]:
pos_embedding_layer = Embedding(len(pos_index)+1,
                                100,
                                input_shape=(N, n_t),
                                trainable=1,
                                #mask_zero=True,
                                #input_length=n_t
                               )

In [28]:
dep_embedding_layer = Embedding(len(dep_index)+1,
                                100,
                                input_shape=(N, n_d),
                                trainable=1,
                                #mask_zero=True
                               )

In [69]:
word_sequence_input = Input(shape=(N, n_w, ), dtype='int32')
word_embedded_sequences = word_embedding_layer(word_sequence_input)

In [70]:
pos_sequence_input = Input(shape=(N, n_t, ), dtype='int32')
pos_embedded_sequences = pos_embedding_layer(pos_sequence_input)

In [72]:
dep_sequence_input = Input(shape=(N, n_d, ), dtype='int32')
dep_embedded_sequences = dep_embedding_layer(dep_sequence_input)

In [73]:
features = Input(shape=(N, n_num,))

In [79]:
word = Reshape((N, -1))(word_embedded_sequences)
pos = Reshape((N, -1))(pos_embedded_sequences)
dep = Reshape((N, -1))(dep_embedded_sequences)
x = concatenate(inputs=[word, pos, dep, features], axis=-1)
x = LSTM(256, return_sequences=True)(x)
x = Dropout(0.3)(x)
preds = Dense(n_unq, activation='softmax',
             kernel_regularizer=regularizers.l2(1e-8))(x)

In [80]:
model = Model(inputs=[word_sequence_input, pos_sequence_input, dep_sequence_input, features], outputs=preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['acc'])

In [81]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 5, 18)        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 5, 18)        0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 5, 12)        0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 5, 18, 300)   7751400     input_5[0][0]                    
__________________________________________________________________________________________________
embedding_

In [427]:
model.fit(
    [X_train[:, :, :n_w], X_train[:, :, n_w:n_w+n_t], X_train[:, :, n_w+n_t:n_w+n_t+n_d], X_train[:, :, n_w+n_t+n_d:]], 
          y_train, 
    validation_data=([X_test[:, :, :n_w], X_test[:, :, n_w:n_w+n_t], X_test[:, :, n_w+n_t:n_w+n_t+n_d], X_test[:, :, n_w+n_t+n_d:]], y_test), 
          epochs=6, 
          batch_size=128, 
          verbose=1)

Train on 32716 samples, validate on 6426 samples
Epoch 1/6
32716/32716 [==============================] - 102s 3ms/step - loss: 0.6187 - acc: 0.8274 - val_loss: 0.3851 - val_acc: 0.8776
Epoch 2/6
32716/32716 [==============================] - 100s 3ms/step - loss: 0.2937 - acc: 0.9093 - val_loss: 0.3546 - val_acc: 0.8837
Epoch 3/6
32716/32716 [==============================] - 100s 3ms/step - loss: 0.2215 - acc: 0.9329 - val_loss: 0.3282 - val_acc: 0.8912
Epoch 4/6
32716/32716 [==============================] - 101s 3ms/step - loss: 0.1763 - acc: 0.9482 - val_loss: 0.3218 - val_acc: 0.8942
Epoch 5/6
32716/32716 [==============================] - 105s 3ms/step - loss: 0.1434 - acc: 0.9599 - val_loss: 0.3205 - val_acc: 0.8943
Epoch 6/6
32716/32716 [==============================] - 111s 3ms/step - loss: 0.1185 - acc: 0.9682 - val_loss: 0.3241 - val_acc: 0.8948


#### Calculate metrics

In [517]:
def LUAS(parser, trees, oracle=None, feature_extractor=None):
    total, tpL, tpU, failed = 0, 0, 0, 0
    for tree in trees:
        golden = [(node["id"], node["head"], node["deprel"]) for node in tree]
        try:
            _, _, predicted, *_ = parser_2.parse(tree, oracle=oracle, update_label_index=False,
                                               feature_extractor=feature_extractor)
            total += len(golden)
            tpL += len(set(golden).intersection(set(predicted)))
            tpU += len(set([(c,h) for c,h,_ in golden]).intersection([(c,h) for c,h,_ in predicted]))
        except:
            failed += 1
    return total, tpL, tpU, failed

In [520]:
total, tpL, tpU, failed = LUAS(parser, test_trees, model, feature_extractor=feature_builder)
print("Failed:", failed)
print("Total:", total)
print("Correctly defined (unlabeled):", tpU)
print("Correctly defined (labeled):", tpL)
print("UAS:", round(tpU / total, 3))
print("LAS:", round(tpL / total, 3))

Failed: 9
Total: 14769
Correctly defined (unlabeled): 11227
Correctly defined (labeled): 10703
UAS: 0.76
LAS: 0.725


In [ ]:
model.save("LSTM.h5")